In [1]:
import json
from ezprettyjson import prettyjson
from deep_dict_utils import *

In [3]:
files = []
for fname in map(lambda s: s+'.json', ['models/ng_catalog'
                                      ,'ml/batch50-nodrop'
                                      ,'ml/batch50-nogauss'
                                      ,'ml/eg_fd-l2-2'
                                      ,'ml/eg_fd-nol2-2'
                                      ,'ml/long-2'
                                      ,'ml/off-2'
                                      ,'ml/on-2'
                                      ,'ml/short-2'
                                      ,'ml/short-batch50'
                                      ,'ml/shorter'
                                      ,'ml/shortest'
                                      ,'ml/short-ratevar']):
    with open(fname, 'r') as f:
        files.append(json.load(f))

In [4]:
all_keys = {k for f in files for k in f.keys()}
lens_per_key = {k: [len(v) for f in files for fk,v in f.iteritems() if fk == k] for k in all_keys}
conflict_keys = map(lambda t: t[0], 
                    filter(lambda (_,v): len(v)>1 and not all(map(lambda e: e==v[0], v)), lens_per_key.iteritems()))
prettyjson({k: [d[k] for d in files if k in d] for k in conflict_keys})

In [7]:
# If conflict_keys is only stuff like e-best and m-latest this is fine
assert set(conflict_keys) == {'e-best', 'e-latest', 'm-best', 'm-latest'}
merged = {k: v for d in files for k,v in d.iteritems() if not k in conflict_keys}

In [8]:
common_keys = ["batch_size", "drop_rate", "epochs", "gaussian_base_sigma", "gaussian_sigma_factor", "l1", "l2", "layer_sizes"]
e_keys = ["enc_use_drop", "enc_use_noise", "encdec_optimizer", "layers", "quality", "train_quality"]
m_keys = ["class_loss", "class_optimizer", "encdecs_name", "labels", "mod_use_drop", "mod_use_noise", "pretrain_end_time", "pretrain_start_time", "test_accuracy", "test_loss"]
c_keys = ["cross_validation", "settings"]
# Some checks
checks = {k: (lambda v: all([trace(trace(x) in v) for x in l]))
          for k,l in [('e-', common_keys + e_keys)
                     ,('m-', common_keys + m_keys)
                     ,('',   c_keys)]}
for k,v in merged.iteritems():
    if "batch50" in k and ("e-" in k or "m-" in k):
        if not "batch_size" in v.keys():
            merged[k]["batch_size"] = 50
        if "m-" in k and not "mod_use_drop" in v.keys():
            merged[k]["mod_use_drop"] = bool(merged[k]["drop_rate"])
            merged[k]["mod_use_noise"] = bool(merged[k]["gaussian_base_sigma"])
    if "short-2" in k and ("e-" in k or "m-" in k):
        if not "batch_size" in v.keys():
            merged[k]["batch_size"] = 100
        if "m-" in k and not "mod_use_drop" in v.keys():
            merged[k]["mod_use_drop"] = bool(merged[k]["drop_rate"])
            merged[k]["mod_use_noise"] = bool(merged[k]["gaussian_base_sigma"])
    if "e-" in k:
        missing = [key for key in common_keys+e_keys if not key in v.keys()]
        if missing:
            print(k+": "+str(missing))
    elif "m-" in k:
        missing = [key for key in common_keys+m_keys if not key in v.keys()]
        if missing:
            print(k+": "+str(missing))
    else:
        missing = [key for key in c_keys if not key in v.keys()]
        if missing:
            print(k+": "+str(missing))
# If everything's alright this prints nothing

In [9]:
prettyjson(merged)

In [10]:
from ml.catalog import CustomJSONEncoder
with open('catalog-modern_only-merged.json', 'w') as f:
    f.write(json.dumps(merged, indent=2, sort_keys=True, cls=CustomJSONEncoder))